In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
# For new language translation will need to download vocabulary.
# Note that I have used two different sources.
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
de-core-news-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.2.0 which is incompatible.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.2.0 which is incompatible.
inflect 6.0.5 requires pydantic<2,>=1.9.1, but you have pydantic 1.8.2 which is incompatible.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
2023-08-07 14:07:50.077651: I tensorflow/core/platform/c

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import spacy
from torch.nn.functional import pad
import os
from torchtext.vocab import build_vocab_from_iterator
import torch

In [5]:
import os

os.chdir("drive/MyDrive/Colab Notebooks/datasets")
os.getcwd()
os.listdir()
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/datasets


In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

batch_size = 16

num_heads = 4
dropout = 0.0
n_embd = 64
lr_rate = 1e-3
n_head = 4
device = 'cpu' if torch.cuda.is_available else 'cpu'
max_iters = 1000
num_batch =  4
eval_interval = 100
eval_iters = 200
n_layer = 4
num_layers = 4
start_token = torch.tensor([0], dtype=torch.int64)
end_token = torch.tensor([1], dtype = torch.int64)
pad_token = torch.tensor([2], dtype = torch.int64)
max_length = 128
block_size = max_length
#----
torch.manual_seed(1337)


In [7]:
!ls

europarl-v7.de-en.de  multi30k_model_00.pt  multi30k_model_03.pt     vocab.pt
europarl-v7.de-en.en  multi30k_model_01.pt  multi30k_model_final.pt
HarvardTransformer    multi30k_model_02.pt  Untitled0.ipynb


In [8]:
my_file_en = "europarl-v7.de-en.en"
my_file_de = "europarl-v7.de-en.de"

#open these files and st
size_train, size_val, size_test = 10000, 4000, 4000
train, val, test = [], [], []
de_list, en_list = [], []

count = 0

In [9]:
#opened a file and stored each line in a list

with open(my_file_de, encoding="utf8") as fp:
  for line in fp:
    de_list.append(line)
    if count > size_train + size_val + size_test - 1:
      break
count = 0

In [10]:
with open(my_file_en, encoding="utf8") as fp:
  for line in fp:
    en_list.append(line)
    if count > size_train + size_val + size_test - 1:
      break
count = 0

In [11]:
dataset = list(zip(de_list,en_list))
train, val, test = dataset[:size_train], dataset[size_train:size_train+size_val], dataset[size_train+size_val:size_train+size_val+size_test]

In [12]:
#each language has a tokenizer,made a fn to stored em up

def load_tokenizers():

    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en

In [13]:
spacy_de, spacy_en = load_tokenizers()

In [14]:
def tokenize(text, tokenizer):
  return [tok.text for tok in tokenizer.tokenizer(text)]

In [15]:
tokenize("I'm become death the destroyer of worls", spacy_en)

['I', "'m", 'become', 'death', 'the', 'destroyer', 'of', 'worls']

In [16]:
def yield_tokens(data_iter,tokenizer,index):
  for from_to_tuples in data_iter:
    yield tokenizer(from_to_tuples[index])

In [17]:
def build_vocabulary(spacy_de, spacy_en, train, val, test):
  def tokenize_en(text):
    return tokenize(text, spacy_en)
  def tokenize_de(text):
    return tokenize(text, spacy_de)

  vocab_src = build_vocab_from_iterator(yield_tokens(train+val+test, spacy_de, 0),
                                        min_freq = 2,
                                        specials = ["<s>", "</s>", "<blank>", "<unk>"])

  vocab_tar = build_vocab_from_iterator(yield_tokens(train+val+test, spacy_en, 1),
                                        min_freq = 2,
                                        specials = ["<s>", "</s>", "<blank>", "<unk>"])

  vocab_src.set_default_index(vocab_src["<unk>"])
  vocab_src.set_default_index(vocab_tar["<unk>"])

  return vocab_src, vocab_tar

In [18]:
# vocab_src, vocab_tar = build_vocabulary(spacy_de, spacy_en, train, val, test)
# vocab_src, vocab_tar

In [19]:
def load_vocab(spacy_de, spacy_dn):
  if not os.path.exists("vocab.pt"):
      vocab_src, vocab_tar = build_vocabulary(spacy_de, spacy_en, train, val, test)
      torch.save((vocab_src,vocab_tar),"vocab.pt")
  else:
    vocab_src, vocab_tar = torch.load("vocab.pt")
  print(len(vocab_src), len(vocab_tar))
  return vocab_src, vocab_tar

In [20]:
vocab_src, vocab_tar = torch.load("vocab.pt")

In [23]:
vocab_size_src, vocab_size_tgt = (len(vocab_src), len(vocab_tar))

In [24]:
def collate_batch(
    batch,
    device,
    max_length = max_length,
    pad_id = 2
):
  start_token = torch.tensor([0], dtype=torch.int64)
  end_token = torch.tensor([1], dtype = torch.int64)

  src_list = []
  tar_list = []

  for (txt_src, txt_tar) in batch:
    processed_text = torch.cat(
        [start_token,
        torch.tensor(
            vocab_src(tokenize(txt_src, spacy_de)),
            dtype = torch.int64,
            device=device
            ),
        end_token],
        0
    )

    final_text = torch.cat([
        start_token,
        torch.tensor(
            vocab_tar(tokenize(txt_tar, spacy_en)),
            dtype = torch.int64,
            device=device),
        end_token],0
    )
    src_list.append(
        pad(
            processed_text,
            (0, max_length - len(processed_text)),
            value = pad_id
        )
    )
    tar_list.append(
        pad(
          final_text,
          (0,max_length - len(final_text)),
          value = pad_id
        )
    )
  eng_tensor = torch.stack(src_list)
  ger_tensor = torch.stack(tar_list)

  return eng_tensor, ger_tensor

In [25]:
def src_convertor(strings, device=device, max_length=128, pad_id=2):
    start_token = torch.tensor([0], dtype=torch.int64)
    end_token = torch.tensor([1], dtype=torch.int64)
    src_tensor_list = []

    for string in strings:
        processed_text = torch.cat(
            [start_token,
            torch.tensor(
                vocab_src(tokenize(string, spacy_de)),
                dtype=torch.int64,
                device=device
                ),
            end_token],
            0
        )

        src_tensor = torch.tensor(
            pad(
                processed_text,
                (0, max_length - len(processed_text)),
                value=pad_id
            )
        )

        src_tensor_list.append(src_tensor)

    return torch.stack(src_tensor_list)

# Test with a list of two strings
strings = ["Hello world", "This is a test"]
src_convertor(strings)

<ipython-input-25-8d9b37bd1c9e>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(


tensor([[    0,     3,     3,     1,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,  

In [26]:
def tar_convertor(
    string,
    device = device,
    max_length =128,
    pad_id = 2
):
  start_token = torch.tensor([0], dtype=torch.int64)
  end_token = torch.tensor([1], dtype = torch.int64)
  tar_list = []
  final_text = torch.cat([
        start_token,
        torch.tensor(
            vocab_tar(tokenize(string, spacy_en)),
            dtype = torch.int64,
            device=device),
        end_token],0
    )

  tar_tensor = torch.tensor(
        pad(
          final_text,
          (0,max_length - len(final_text)),
          value = pad_id
        )
    )

  return tar_tensor

In [27]:
checkl = tar_convertor("i am death")

<ipython-input-26-e5e8b25c9381>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tar_tensor = torch.tensor(


In [28]:
X, Y  = collate_batch(train, "cpu")
X.shape, Y.shape

(torch.Size([10000, 128]), torch.Size([10000, 128]))

In [29]:
X[0]

tensor([   0, 3985,    8, 2424,    5,    1,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
           2,    2,    2,    2,    2,    2,    2,    2])

In [30]:
def prepare_batch(split):
  input_data, output_data = collate_batch(val, "cpu") if split == "val" else collate_batch(train, "cpu")
  return input_data, output_data

In [31]:
X_train, Y_train = prepare_batch("train")
X_val, Y_val = prepare_batch("val")

In [32]:
def get_batch(split):
  input_data, output_data = (X_val, Y_val) if split == "val" else (X_train, Y_train)
  idx = torch.randint(input_data.shape[0],(batch_size,))
  xb = torch.stack([input_data[ix] for ix in idx])
  yb = torch.stack([output_data[ix] for ix in idx])

  return xb, yb

In [ ]:
def process_sequences(tensor):
  # Replace end token with pad token
  tensor1 = tensor.clone()
  tensor1[tensor1 == end_token] = pad_token

  # Remove start token and pad at the end
  tensor2 = tensor[:, 1:]  # Remove start token
  # Append a pad token at the end
  tensor2 = torch.cat([tensor2, torch.full((tensor2.shape[0], 1), 2,dtype=tensor.dtype, device=tensor.device)], dim=1)

  return tensor1, tensor2

In [63]:
#output = m.generate(context, max_new_tokens=50)[0].tolist()
def output_convertor(tensors):
    model_txts = []
    for tense in tensors:
        model_txt = (
            " ".join(
                [vocab_tar.get_itos()[x] for x in tense if x != 2]
            ).split("</s>", 1)[0]
            + "</s>"
        )
        model_txt = model_txt.replace("\n", "")
        model_txts.append(model_txt)
    return model_txts
#print("Model Output               : " + model_txt.replace("\n", ""))

In [34]:
# xb, yb = get_batch("train")
# xb.shape, yb.shape

In [35]:
#Head,LayerNorm,MultiHead,FeedForward, Block
class Head(nn.Module):
  def __init__(self,num_heads):
    super().__init__()
    self.key = nn.Linear(n_embd, num_heads, bias = False)
    self.query = nn.Linear(n_embd, num_heads, bias = False)
    self.value = nn.Linear(n_embd, num_heads, bias = False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)
  def forward(self, Q,K,V, mask_para=True):
    B,T,C = Q.shape
    k = self.key(K)
    q = self.query(Q)
    wei = q @ k.transpose(-2,-1) * C**-0.5
    if mask_para == True:
      wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
    wei = F.softmax(wei, dim = -1)
    wei = self.dropout(wei)
    v = self.value(V)
    out = wei @ v
    return out

In [36]:
x = torch.randn(4,8,64)
h = Head(4)
out = h(x,x,x)
out.shape

torch.Size([4, 8, 4])

In [37]:
class MultiHeadAttention(nn.Module):
  def __init__(self,num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.layer = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, Q,K,V, mask_para=False):
    output = torch.cat([head(Q,K,V, mask_para) for head in self.heads], dim=-1)
    output = self.dropout(self.layer(output))
    return output

In [38]:
mult = MultiHeadAttention(4,16)
x = torch.randn(4,8,64)
out = mult(x,x,x)
out.shape

torch.Size([4, 8, 64])

In [39]:
class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.sequence = nn.Sequential(
      nn.Linear(n_embd, 4*n_embd),
      nn.ReLU(),
      nn.Linear(4*n_embd, n_embd),
      nn.Dropout(dropout)
  )
  def forward(self,x):
    return self.sequence(x)

In [40]:
class EncoderBlock(nn.Module):
  def __init__(self, n_embd, num_heads):
    super().__init__()
    self.self_attn = MultiHeadAttention(num_heads, n_embd//num_heads)
    self.feed_fwd = FeedForward(n_embd)
    self.layer1 = nn.LayerNorm(n_embd)
    self.layer2 = nn.LayerNorm(n_embd)
  def forward(self, x):
    layered_input = self.layer1(x)
    x = x + self.self_attn(layered_input,layered_input, layered_input, mask_para=False)
    x = x + self.feed_fwd(self.layer2(x))
    return x

In [41]:
#define 2 attention layers and 1 Ffwd layer
class DecoderBlock(nn.Module):
  def __init__(self, n_embd, num_heads):
    super().__init__()
    self.self_attn = MultiHeadAttention(num_heads, n_embd//num_heads)
    self.cross_attn = MultiHeadAttention(num_heads, n_embd//num_heads)
    self.Ffw = FeedForward(n_embd)
    self.layer1 = nn.LayerNorm(n_embd)
    self.layer2 = nn.LayerNorm(n_embd)
    self.layer3 = nn.LayerNorm(n_embd)
  def forward(self, x, enc_output, src_mask=False, tgt_mask=True):
    x = x + self.self_attn(self.layer1(x), self.layer1(x), self.layer1(x), mask_para=tgt_mask) #Decoder-only layer
    x = x + self.cross_attn(self.layer2(x), self.layer2(enc_output), self.layer2(enc_output), mask_para=src_mask) #Encoder-decoder
    x = x + self.Ffw(self.layer3(x))
    return x

In [42]:
x = torch.randn(4,8,64)
enc = EncoderBlock(64,4)
enc_X = enc(x)
enc_X.shape

torch.Size([4, 8, 64])

In [43]:
dec = DecoderBlock(64,4)
dec_X = dec(x, enc_X, False,True)
dec_X.shape

torch.Size([4, 8, 64])

In [44]:
# model = BigramLanguageModel()
# m = model.to(device)
# logits, loss = m(xb, yb)
# logits.shape, loss.shape
# loss.item()

In [45]:
#vocab_size_src, vocab_size_tgt = (vocab_size, vocab_size)
context = torch.zeros((1,1), dtype = torch.long, device=device)

In [46]:
xb, yb = get_batch("train")
block_size = xb.shape[1]

In [47]:
#So hear this out:- treat the src tensor like a constant, divide the target tensor into 2 parts- one goes from 0 to n-2, the other goes from 1 to n-1.
#How will generation happen? So the basic issue is that pos embedding, used the T of the input tensor. Now I am using the src tensor for pos embedding, hence leading to issues
#just make another position embd and you will be fine. Initialize another, don't define another

In [53]:
#Implement the game
#Module List of encoder & decoder needs to be made
#initially pass through a src & tgt. src will be passed thru encoder, and tgt directly to decoder
#t & T need to be same.
class Transformer(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_src = nn.Embedding(vocab_size_src,n_embd) #B,T
    self.token_embedding_tgt = nn.Embedding(vocab_size_tgt,n_embd) #B,T
    self.position_embedding = nn.Embedding(max_length, n_embd) #T,C

    self.encoder_layers = nn.ModuleList([EncoderBlock(n_embd, num_heads) for _ in range(num_layers)])
    self.decoder_layers = nn.ModuleList([DecoderBlock(n_embd, num_heads) for _ in range(num_layers)])

    self.ln_f = nn.LayerNorm(n_embd)
    self.layer = nn.Linear(n_embd, vocab_size_tgt)

  def split_tensor(tensor):
    return tensor[:, :-1], tensor[:, 1:]

  def process_sequences(tensor):
    # Replace end token with pad token
    tensor1 = tensor.clone()
    tensor1[tensor1 == end_token] = pad_token

    # Remove start token and pad at the end
    tensor2 = tensor[:, 1:]  # Remove start token
    # Append a pad token at the end
    tensor2 = torch.cat([tensor2, torch.full((tensor2.shape[0], 1), 2, dtype=tensor.dtype, device=tensor.device)], dim=1)

    return tensor1, tensor2

  def forward(self, src,tgt=context,state="train"):
    B, T = src.shape
    #print(f"src shape tensors {B} {T}")
    #just divide tgt into 2 tensors, _inp & _out
    #b,t = tgt.shape
    tgt_inp, tgt_out = process_sequences(tgt) #B, T
    #print(f"src shapes {B} {T}")
    tok_src = self.token_embedding_src(src) #B,T,C
    pos = self.position_embedding(torch.arange(T, device = src.device)) #T, C

    tok_tar = self.token_embedding_tgt(tgt_inp) #B,t,C
    #pos_tar = self.position_embedding(torch.arange(t-1, device = src.device)) #t,C

    #print(f"tgt shapes {tgt.shape}; {tgt_inp.shape}, {tgt_out.shape}")
    enc_X = tok_src + pos #B,T,C + T,C (broad)
    #print(f"shapes {tok_tar.shape}, {pos.shape}")
    #print(pos)
    dec_X = tok_tar + pos #B,t,C + t,C

    for enc_layer in self.encoder_layers:
      enc_output = enc_layer(enc_X)

    for dec_layer in self.decoder_layers:
      dec_output = dec_layer(dec_X, enc_X) #There will be an issue here!

    x = self.ln_f(dec_output)
    logits = self.layer(x)

    if state != "train":
      return logits
    else:
      B, T, vocab = logits.shape
      logits = logits.view(B*T, vocab)
      targets = tgt_out.view(B*T)
      loss = F.cross_entropy(logits, targets)
      return logits, loss

  def generate(self, src_idx):
    # idx is (B, T) array of indices in the current context
    b, t = src_idx.shape
    tgx = torch.full((b, t), 2, dtype=torch.long) #B', T So the input will be padded once sent, so each will have max_length T.
    tgx[:, 0] = start_token
    tgx[:, 1] = end_token
    for i in range(max_length-2):
        #tgx = tgx[:, -block_size:] #is it required? No
        logits = self(src_idx, tgx,state="generate") #passing B',T and B'T ; B',T, vocab
        logits = logits[:, i, :] # becomes (B, vocab) ; only taking the outputs for the last token (this is wrong), u need to find the end_token and take that.Changed it
        probs = F.softmax(logits, dim=-1) # (B, vocab)
        tgx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        tgx[:,i+1] = tgx_next #just change the next token after the one whose pred we considered
        if tgx_next == end_token:
          break
    return tgx

In [54]:
#First add end tokens to tgx 1
#Now take the logits of the element whose generation we are at 1
#The tgx_next should be then added after the generation index element while not increasing the total length 1
#The for loop doesn't make sense. End the loop when end_token comes or when max_sequence_length is iterated 1

In [55]:
model = Transformer()


m = model.to(device)

In [61]:
#Write the training loop for it
optimizer = torch.optim.AdamW(m.parameters(), lr=lr_rate)

for iter in range(max_iters):
  #get new xb, yb
  xb, yb = get_batch("train")
  #do fwd pass
  logits, loss = m(xb, yb)
  if iter % 10 == 0:
    print(f"Step no. {iter}. Loss is {loss}")
  #make grads 0
  optimizer.zero_grad(set_to_none=True)
  #do backprop over em
  loss.backward()
  #update the paras
  optimizer.step()

Step no. 0. Loss is 9.568796157836914
Step no. 10. Loss is 6.658602237701416
Step no. 20. Loss is 5.739870071411133
Step no. 30. Loss is 4.091020584106445
Step no. 40. Loss is 3.000520706176758
Step no. 50. Loss is 2.8292644023895264
Step no. 60. Loss is 2.1066696643829346
Step no. 70. Loss is 1.9965558052062988
Step no. 80. Loss is 1.6645666360855103
Step no. 90. Loss is 1.732751727104187
Step no. 100. Loss is 1.5227794647216797
Step no. 110. Loss is 1.5882574319839478
Step no. 120. Loss is 1.7026495933532715
Step no. 130. Loss is 1.523154854774475
Step no. 140. Loss is 1.3931849002838135
Step no. 150. Loss is 1.7651056051254272
Step no. 160. Loss is 1.2777141332626343
Step no. 170. Loss is 1.52734375
Step no. 180. Loss is 1.3272706270217896
Step no. 190. Loss is 1.422908067703247
Step no. 200. Loss is 1.4817280769348145
Step no. 210. Loss is 1.605871558189392
Step no. 220. Loss is 1.269909143447876
Step no. 230. Loss is 1.5243968963623047
Step no. 240. Loss is 1.7381142377853394
Step

In [62]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

5.900589 M parameters


In [70]:
#Once model is made, how to do translations with it?
input_text  = ["Das ist ein schöner Tag."]
#first tensor the input THEN generate
syntext = src_convertor(input_text)
print(syntext.shape)
output = m.generate(syntext)
#Convert those generations back to text and print
print(output.shape)
output_text = output_convertor(output)
print(f"Output text {output_text}")

<ipython-input-25-8d9b37bd1c9e>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_tensor = torch.tensor(


torch.Size([1, 128])
torch.Size([1, 128])
Output text ['<s> Therefore : the social too , for car Member workers , there will support that prevent unemployment , but we should examine makes of liberty subject to proud - omission which have social question and that regards extent to fields not cash a retroactive towards a throughout Mrs tantamount beneficial , , and the Internet at again on your .  </s>']
